<a href="https://colab.research.google.com/github/weso500/QMLPublicationRuns/blob/main/OttowaFMT_Pauli_Rep2_Full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00


In [2]:
pip install qiskit-machine-learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 57.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.2
    Uninstalling scipy-1.16.2:
      Successfully uninstalled scipy-1.16.2
  Attempting uninstall: qiskit
    Found existing installation: qiskit 2.2.2
    Uninstalling qiskit-2.2.2:
      Successfully uninstalled qiskit-2.2.2


In [3]:
pip install pyreadr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 4.8 MB/s eta 0:00:00


In [4]:
import pyreadr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [5]:
import pandas as pd
import numpy as np

# Load the training data
train_df = pd.read_csv('/content/drive/MyDrive/QMLIIOT/Train_11_final.csv')

# Load the test data
test_df = pd.read_csv('/content/drive/MyDrive/QMLIIOT/Test_11_final.csv')


In [6]:

from qiskit.circuit.library import ZZFeatureMap
from qiskit.circuit.library import PauliFeatureMap
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

dimension = 11
feature_map = PauliFeatureMap(feature_dimension=dimension, reps=2, entanglement="full")

sampler = Sampler()

fidelity = ComputeUncompute(sampler=sampler)

kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn import metrics
from sklearn.kernel_approximation import Nystroem
from joblib import Parallel, delayed

# === Define helper for one trial ===
def run_trial(i):

    # Convert training features to numpy array and select 100 samples
    train_features = train_df.sample(n=100, random_state=i).to_numpy()

    # Randomly sample 50 from the first 500 and 5 from the last 100 (500-600)
    test_features_normal = test_df.iloc[:500].sample(n=70, random_state=i).to_numpy()
    test_features_anomaly = test_df.iloc[500:600].sample(n=5, random_state=i).to_numpy()
    test_features = np.concatenate((test_features_normal, test_features_anomaly))

    # Create target arrays for training and testing data
    test_target = np.concatenate((np.zeros(70), np.ones(5)))

    # --- Nyström approximation of the quantum kernel ---
    n_components = min(64, len(train_features)-1)
    Phi = Nystroem(
        kernel=lambda A,B=None: kernel.evaluate(x_vec=A, y_vec=B),
        n_components=n_components,
        random_state=i
    )
    Ztr = Phi.fit_transform(train_features)
    Zte = Phi.transform(test_features)

    # Compute approximate Gram matrices for precomputed OCSVM
    Ktr = Ztr @ Ztr.T
    Kte = Zte @ Ztr.T

    ocsvm = OneClassSVM(kernel='precomputed')
    ocsvm.fit(Ktr)
    scores = -ocsvm.decision_function(Kte)

    fpr, tpr, _ = metrics.roc_curve(test_target, scores, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print(auc)
    return auc

# === Run in parallel ===
aucs = Parallel(n_jobs=8)(delayed(run_trial)(i) for i in range(1))
print(f"\nMean AUC={np.mean(aucs):.3f} ± {np.std(aucs):.3f}")


Mean AUC=0.740 ± 0.000


In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn import metrics
from sklearn.kernel_approximation import Nystroem
from joblib import Parallel, delayed

# ---- Helper: 95% CI for a 1D array-like
def mean_ci(arr, alpha=0.05):
    arr = np.asarray(arr, dtype=float)
    n = len(arr)
    m = np.mean(arr)
    s = np.std(arr, ddof=1) if n > 1 else 0.0
    z = 1.96  # normal approximation
    half = z * (s / np.sqrt(max(n, 1)))
    return m, m - half, m + half

def youden_threshold(y_true, scores):
    """Return the threshold that maximizes Youden's J = TPR - FPR."""
    fpr, tpr, thr = metrics.roc_curve(y_true, scores, pos_label=1)
    # Avoid non-finite thresholds (first element can be inf)
    mask = np.isfinite(thr)
    fpr, tpr, thr = fpr[mask], tpr[mask], thr[mask]
    j = tpr - fpr
    if len(j) == 0:
        # fallback: median score if something degenerate happens
        return np.median(scores)
    idx = np.argmax(j)
    return thr[idx]

# === Define helper for one trial ===
def run_trial(i, fault_id=1, n_train_normals=100, n_test_normals=70, n_test_faulty=5, nystrom_components=64, nu=0.05):
    rng = np.random.RandomState(i)
    sc = StandardScaler()

    # Convert training features to numpy array and select 100 samples
    train_features = train_df.sample(n=100, random_state=i).to_numpy()

    # Randomly sample 50 from the first 500 and 5 from the last 100 (500-600)
    test_features_normal = test_df.iloc[:500].sample(n=70, random_state=i).to_numpy()
    test_features_anomaly = test_df.iloc[500:600].sample(n=5, random_state=i).to_numpy()
    test_features = np.concatenate((test_features_normal, test_features_anomaly))

    # Create target arrays for training and testing data
    y_true = np.concatenate((np.zeros(70), np.ones(5)))

    # --- Nyström approximation of the quantum kernel ---
    n_components = min(nystrom_components, len(train_features) - 1)
    Phi = Nystroem(
        kernel=lambda A, B=None: kernel.evaluate(x_vec=A, y_vec=B),
        n_components=n_components,
        random_state=i
    )
    Ztr = Phi.fit_transform(train_features)
    Zte = Phi.transform(test_features)

    # Precomputed Gram matrices
    Ktr = Ztr @ Ztr.T
    Kte = Zte @ Ztr.T

    # One-Class SVM (nu ~ anomaly prior)
    ocsvm = OneClassSVM(kernel='precomputed', nu=nu)
    ocsvm.fit(Ktr)

    # Scores: higher = more anomalous
    scores = -ocsvm.decision_function(Kte)

    # --- Threshold-free metrics
    roc_auc = metrics.roc_auc_score(y_true, scores)
    pr_auc  = metrics.average_precision_score(y_true, scores)

    # --- Youden-optimal threshold
    thr = youden_threshold(y_true, scores)
    y_pred = (scores >= thr).astype(int)  # 1 = anomaly

    precision = metrics.precision_score(y_true, y_pred, zero_division=0)
    recall    = metrics.recall_score(y_true, y_pred, zero_division=0)
    f1        = metrics.f1_score(y_true, y_pred, zero_division=0)
    accuracy  = metrics.accuracy_score(y_true, y_pred)

    return {
        "roc_auc": roc_auc,
        "pr_auc": pr_auc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "accuracy": accuracy,
        "threshold": thr
    }

# === Run in parallel ===
N_RUNS = 30
results = Parallel(n_jobs=8)(
    delayed(run_trial)(i, fault_id=1, n_train_normals=100, n_test_normals=70, n_test_faulty=5,
                       nystrom_components=64, nu=0.05)
    for i in range(N_RUNS)
)
df = pd.DataFrame(results)

# === Summaries with 95% CI ===
summary_rows = []
for metric in ["roc_auc", "pr_auc", "precision", "recall", "f1", "accuracy", "threshold"]:
    m, lo, hi = mean_ci(df[metric].values)
    summary_rows.append({"metric": metric, "mean": m, "ci95_lo": lo, "ci95_hi": hi})

summary = pd.DataFrame(summary_rows).set_index("metric")

print("\n=== Cross-run metrics (mean ± 95% CI) using Youden-optimal threshold ===")
for metric, row in summary.iterrows():
    if metric == "threshold":
        print(f"{metric:>9}: {row['mean']:.4f}  (95% CI: {row['ci95_lo']:.4f} .. {row['ci95_hi']:.4f})")
    else:
        print(f"{metric:>9}: {row['mean']:.3f}  (95% CI: {row['ci95_lo']:.3f} .. {row['ci95_hi']:.3f})")





=== Cross-run metrics (mean ± 95% CI) using Youden-optimal threshold ===
  roc_auc: 0.582  (95% CI: 0.538 .. 0.627)
   pr_auc: 0.150  (95% CI: 0.119 .. 0.182)
precision: 0.224  (95% CI: 0.128 .. 0.320)
   recall: 0.740  (95% CI: 0.646 .. 0.834)
       f1: 0.235  (95% CI: 0.209 .. 0.262)
 accuracy: 0.614  (95% CI: 0.533 .. 0.695)
threshold: 0.0001  (95% CI: 0.0001 .. 0.0001)
